# Подготовка датасета

In [133]:
# Подключаем библиотеки

# Для парсинга файла html
from bs4 import BeautifulSoup 
import re
import os
import fnmatch
import numpy as np

# Создаем список, элементы которого будут представлять текст статьи на опредленную тему. 
collection = []

# Всего областей 5: a) биология, b) информационные технологии, c) история, d) менеджмент, 
# e) архитектура и строительство
# Создадим список, в котором каждому номеру документа соответствует область
field = []


# Читаем все файлы из папки dataset
for filename in os.listdir("dataset"):
    with open(os.path.join("dataset", filename), "rb") as f:
        contents = f.read().decode(errors='replace')
        soup = BeautifulSoup(contents, 'html.parser')
        
        a = soup.find_all('p', style = re.compile("[\D+]?text-align: justify;"))
        s = ''
        
        for i in range(len(a)):
        
# Некоторые атрибуты в файле имели пустые значения и при записи почему-то записывались как \xa0, поэтому
# пришлось проводить проверку на то, чтобы a[i].text != '\xa0' и только потом записывать в общую строку
            if a[i].text != '\xa0':
                s += ' ' + re.sub('[a-zA-Z0-9%-.,:—;"!№/\[\]\\\'?»«–’”“℃]','',a[i].text.lower())
         
    
# При парсинге не получалось удалить некоторые объекты, пришлось ручками.
    s = s.replace('\u200b','')
    s = s.replace('\xad', '')
    s = s.replace('\xa0', '')
    collection.append(s)
    
    if fnmatch.fnmatch(filename, 'a*.html'):
        field.append('Биология')
    elif fnmatch.fnmatch(filename, 'b*.html'):
        field.append('Информационные технологии')
    elif fnmatch.fnmatch(filename, 'c*.html'):
        field.append('История')
    elif fnmatch.fnmatch(filename, 'd*.html'):
        field.append('Менеджмент')
    elif fnmatch.fnmatch(filename, 'e*.html'):
        field.append('Архитектура')
    else: field.append('1')
    
collection.remove(collection[14])

for i in range(len(field)-1):
    if field[i] == '1':
        field.remove(field[i])

# field

# Мощность датасета и пример одного элемента 
# print(len(collection))
# print(collection[108])

In [134]:
collection_split = [collection[i].split() for i in range(len(collection))]

data = []
for i in range(len(collection_split)):
    data.append(list(filter(lambda x: len(x) > 4, collection_split[i])))
# data[108]

In [135]:
# Напишем функцию для лемматизации текста и выведем коллекцию статей снова

import pymorphy2

morph = pymorphy2.MorphAnalyzer()

def lemmat(data):
    res_data = []
    for words in data:
        res_line = []
        for word in words:
            res_line.append(morph.parse(word)[0].normal_form)
        res_data.append(' '.join(res_line)) 
    return res_data

collection = lemmat(data)

In [136]:
data = [collection[i].split() for i in range(len(collection))] 
# data

# Построение модели

In [137]:
# !pip install -U gensim

In [138]:
from gensim.models import Word2Vec

In [139]:
word2vec = Word2Vec(data, min_count = 1)

In [140]:
# Список всех уникальных слов

vocabulary = word2vec.wv.key_to_index
# print(vocabulary)
len(vocabulary)

8463

In [141]:
# Пример вектора для слова
# word2vec.wv['реформа']

In [142]:
# Просто решила посмотреть, как модель определяет сходие по значению слова :)
# word2vec.wv.most_similar('урбанизм')

In [143]:
# Теперь векторизуем каждый документ. Усредним оценки для всех слов в предложении

def vec_w2v(model, text):
    sum_ = 0
    for j in range(len(text)):
        sum_ += model.wv[text[j]]
    return sum_ / len(text)

In [144]:
# vec_w2v(data[1])

# Близость документов

In [145]:
# Напишем функцию, высчитывающую косинусное расстояние

def cos(vec1, vec2):
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    return np.dot(vec1, vec2) / np.linalg.norm(vec1) / np.linalg.norm(vec2)

# Пример 1

In [146]:
# Введем запрос, преобразуем его и подготовим список слов запроса

search = 'Программное обеспечение'
search = lemmat([search.lower().split()])[0].split()
search

['программный', 'обеспечение']

In [147]:
vec_search = vec_w2v(word2vec, search)
# vec_search

In [148]:
# Построим список косинусного расстояния между вектором запроса и векторами статей

scalar = {}
for i in range(len(data)):
    scalar[i] = cos(vec_w2v(word2vec, data[i]), vec_search)

# scalar

In [149]:
# Создание словаря с номерами статей и соответствующими косинусными расстояниями. 
# Словарь сортируется по возрастанию косинусного расстояния. А выводит в обратном порядке, для наглядности.

rang_index = {}

for i in range(len(scalar)):
    rang_index[i] = scalar[i]

list_rang_index = list(rang_index.items())
list_rang_index.sort(key = lambda i: i[1])

for index in reversed(list_rang_index):
    print(index[0], ':', index[1])

101 : 0.9995487
33 : 0.9995477
6 : 0.9995477
29 : 0.9995469
9 : 0.9995469
1 : 0.99954635
5 : 0.9995456
7 : 0.99954545
19 : 0.9995419
69 : 0.9995368
23 : 0.9995357
4 : 0.9995357
56 : 0.9995352
17 : 0.99953496
31 : 0.99953467
38 : 0.9995324
27 : 0.9995316
86 : 0.9995315
89 : 0.99953127
97 : 0.9995311
94 : 0.99953085
68 : 0.99953085
62 : 0.99953085
99 : 0.9995305
3 : 0.9995277
57 : 0.9995275
0 : 0.99952734
34 : 0.9995258
16 : 0.9995258
58 : 0.9995254
10 : 0.99952483
42 : 0.9995237
18 : 0.9995237
78 : 0.9995232
13 : 0.999523
83 : 0.9995227
32 : 0.99952215
84 : 0.9995211
75 : 0.99952096
102 : 0.9995207
95 : 0.99952066
46 : 0.99951994
37 : 0.9995199
12 : 0.9995199
76 : 0.9995188
43 : 0.99951875
14 : 0.9995167
20 : 0.99951583
91 : 0.9995158
106 : 0.9995153
35 : 0.99951524
103 : 0.9995148
77 : 0.9995145
52 : 0.9995145
45 : 0.99951446
63 : 0.99951345
104 : 0.99951315
98 : 0.9995131
48 : 0.9995124
79 : 0.9995122
47 : 0.999511
88 : 0.9995099
108 : 0.99950904
71 : 0.99950904
80 : 0.99950796
55 : 0

In [150]:
# Выведем первые 10 подходящих статей. Я вывожу области, из которых взяты статьи, чтобы предварительно показать
# качество работы модели. Вывод самих статей закомментирован. Чтобы посмотреть - раскомментируйте 7ю строку.

for index in reversed(list_rang_index[-10:]):
        print(field[index[0]])
#         print(collection[index[0]])
#         print('\n')

Информационные технологии


Информационные технологии


Информационные технологии


Информационные технологии


Информационные технологии


Менеджмент


Информационные технологии


Биология


Менеджмент


Информационные технологии




# Пример 2

Вариант, в случае, если в запросе присутствует слово, которое модель не знает

In [151]:
# Введем запрос и преобразуем его
search2 = 'Виды покрытосеменных растений'
search2 = lemmat([search2.lower().split()])[0].split()
search2

['вид', 'покрытосеменной', 'растение']

In [152]:
data.append(search2)

In [153]:
# data[-2:]

In [154]:
new_word2vec = Word2Vec(data, min_count = 1)

In [155]:
vec_search2 = vec_w2v(new_word2vec, search2)
# vec_search

In [156]:
# Построим список косинусного расстояния между вектором запроса и векторами статей

scalar = []
for i in range(len(data)):
    scalar.append(cos(vec_w2v(new_word2vec, data[i]), vec_search2))

# scalar

In [157]:
# Создание словаря с номерами статей и соответствующими косинусными расстояниями. 
# Словарь сортируется по возрастанию косинусного расстояния. А выводит в обратном порядке, для наглядности.

rang_index = {}

for i in range(len(scalar)):
    rang_index[i] = scalar[i]

list_rang_index = list(rang_index.items())
list_rang_index.sort(key = lambda i: i[1])

for index in reversed(list_rang_index):
    print(index[0], ':', index[1])

109 : 1.0
82 : 0.9997046
14 : 0.999702
105 : 0.99969494
41 : 0.99968725
56 : 0.9996852
72 : 0.9996844
106 : 0.99968404
40 : 0.9996824
104 : 0.99968207
74 : 0.9996809
25 : 0.9996804
50 : 0.9996797
78 : 0.9996795
43 : 0.9996782
86 : 0.99967736
20 : 0.9996769
7 : 0.9996765
90 : 0.99967563
103 : 0.9996739
15 : 0.9996725
76 : 0.9996723
101 : 0.99967223
17 : 0.9996713
89 : 0.99967116
26 : 0.9996711
100 : 0.99967057
27 : 0.99967045
96 : 0.99966985
0 : 0.9996688
80 : 0.9996687
55 : 0.9996687
97 : 0.99966824
45 : 0.9996678
49 : 0.9996676
2 : 0.9996671
94 : 0.9996663
68 : 0.9996663
21 : 0.9996661
85 : 0.999666
47 : 0.999666
36 : 0.9996659
77 : 0.9996658
52 : 0.9996658
102 : 0.99966574
108 : 0.9996656
71 : 0.9996656
92 : 0.99966544
91 : 0.99966544
58 : 0.99966544
48 : 0.99966526
93 : 0.9996651
34 : 0.9996647
16 : 0.9996647
70 : 0.99966407
8 : 0.99966407
28 : 0.9996638
64 : 0.9996637
44 : 0.9996632
35 : 0.99966305
98 : 0.9996629
13 : 0.9996627
42 : 0.9996626
18 : 0.9996626
73 : 0.9996624
53 : 0.99

In [158]:
# Выведем первые 10 подходящих статей. Я вывожу области, из которых взяты статьи, чтобы предварительно показать
# качество работы модели. Вывод самих статей закомментирован. Чтобы посмотреть - раскомментируйте 7ю строку.
field.append('Сам запрос')

res_top20 = []
for index in reversed(list_rang_index[-21:-1]):
        print(field[index[0]])
        res_top20.append(index[0])
        print(index[0])
#         print(collection[index[0]])
#         print('\n')
    


Биология
82
Биология
14
Биология
105
Биология
41
Архитектура
56
Биология
72
Менеджмент
106
Архитектура
40
Информационные технологии
104
Архитектура
74
Биология
25
Биология
50
Архитектура
78
История
43
Менеджмент
86
Менеджмент
20
Биология
7
Информационные технологии
90
Биология
103
История
15


In [159]:
data.pop(len(data)-1)
field.pop(len(field)-1)

'Сам запрос'

# Подготовка датасета для оценки качества

Оценка качества модели производится следующим образом:
1. Строится новый список документов по запросу, содержащему синонимы к главным словам
2. Строится список из первых 20 подходящих статей 
3. Список сравнивается со списком первых 20 статей по изходному запросу
4. Считается доля входящих статей первого списка во второй.

In [160]:
# Введем запрос и преобразуем его
search_acr = 'Вид тип разновидность семейство покрытосеменных растений трава цветы и животных скот зверь'
search_acr = lemmat([search_acr.lower().split()])[0].split()
search_acr

['вид',
 'тип',
 'разновидность',
 'семейство',
 'покрытосеменной',
 'растение',
 'трава',
 'цветок',
 'и',
 'животное',
 'скот',
 'зверь']

In [161]:
data.append(search_acr)

In [162]:
new_word2vec_acr = Word2Vec(data, min_count = 1)

In [163]:
vec_search_acr = vec_w2v(new_word2vec_acr, search_acr)

In [164]:
scalar_acr = []
for i in range(len(data)):
    scalar_acr.append(cos(vec_w2v(new_word2vec_acr, data[i]), vec_search_acr))

In [165]:
rang_index = {}

for i in range(len(scalar_acr)):
    rang_index[i] = scalar_acr[i]

list_rang_index = list(rang_index.items())
list_rang_index.sort(key = lambda i: i[1])

for index in reversed(list_rang_index):
    print(index[0], ':', index[1])

109 : 1.0
82 : 0.9997423
25 : 0.99974144
50 : 0.9997342
105 : 0.99973327
49 : 0.9997329
73 : 0.9997303
75 : 0.9997263
42 : 0.99972516
18 : 0.99972516
107 : 0.9997237
45 : 0.99972165
14 : 0.99972045
43 : 0.9997191
26 : 0.9997177
85 : 0.9997163
76 : 0.99971604
30 : 0.99971503
22 : 0.99971485
89 : 0.9997145
60 : 0.9997141
70 : 0.9997112
65 : 0.9997112
62 : 0.99971104
2 : 0.999711
17 : 0.99971086
8 : 0.9997107
54 : 0.99970955
81 : 0.9997091
59 : 0.9997089
56 : 0.9997086
104 : 0.99970835
101 : 0.99970835
91 : 0.9997079
72 : 0.9997076
40 : 0.9997072
53 : 0.99970686
24 : 0.9997068
103 : 0.99970543
63 : 0.99970543
100 : 0.99970514
36 : 0.9997051
102 : 0.9997049
94 : 0.99970466
68 : 0.99970466
35 : 0.99970466
48 : 0.9997043
0 : 0.9997039
41 : 0.99970335
7 : 0.999703
69 : 0.99970275
86 : 0.9997027
20 : 0.9997017
1 : 0.99970084
98 : 0.9997006
97 : 0.9997
99 : 0.9996993
78 : 0.99969923
92 : 0.9996986
38 : 0.99969846
106 : 0.99969804
67 : 0.9996975
61 : 0.9996974
5 : 0.99969715
29 : 0.99969673
9 : 

In [166]:
field.append('Сам запрос')
res_acr = []

for index in reversed(list_rang_index[-21:-1]):
        res_acr.append(index[0])
res_acr

[82,
 25,
 50,
 105,
 49,
 73,
 75,
 42,
 18,
 107,
 45,
 14,
 43,
 26,
 85,
 76,
 30,
 22,
 89,
 60]

In [167]:
data.pop(len(data)-1)
field.pop(len(field)-1)

'Сам запрос'

# Оценка качества модели

In [168]:
k = sum([1.0 for i in res_top20 if i in res_acr])
k/20

0.3

Вывод: Из 20-ти предположительно верных статьи только 6 выявила модель. Что, конечно, не очень хорошо.